##  Data set validation

In [1]:
# First check that the user in Data_User is the same as the user in Data_Action
# Ensure that the actions generated in the behaviour data are generated by the user in the user data 
                                  #(but there may be users who have no actions in the behaviour data)
    

Methods: Use pd.Merge to join two DataFrames based on sku, and see if the data decreases:

In [2]:
import pandas as pd
df1 = pd.DataFrame({'sku':['a','b','c','d'],'data':[1,1,2,3]})
df2 = pd.DataFrame({'sku':['a','b','f']})
df3 = pd.DataFrame({'sku':['a','b','d']})
df4 = pd.DataFrame({'sku':['a','b','c','d']})
display(pd.merge(df1,df2))
display(pd.merge(df1,df3))
display(pd.merge(df1,df4))  # No reduction in index consistency


,sku,data
0,a,1
1,b,1


,sku,data
0,a,1
1,b,1
2,d,3


,sku,data
0,a,1
1,b,1
2,c,2
3,d,3


Check the data

In [3]:
df_user = pd.read_csv('data/JData_User.csv')
display(df_user.head())
df_month3 = pd.read_csv('data/JData_Action_201603.csv')
df_month3.head()

,user_id,age,sex,user_lv_cd,user_reg_tm
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26


,user_id,sku_id,time,model_id,type,cate,brand
0,280567,167208,2016-02-29 23:59:01,0.0,6,4,519
1,270248,35533,2016-02-29 23:59:02,111.0,6,4,306
2,203360,78694,2016-02-29 23:59:02,NaN,1,8,244
3,252369,90402,2016-02-29 23:59:03,0.0,6,7,38
4,279590,154208,2016-02-29 23:59:03,0.0,6,5,570


Recycling rubbish data  (still have !!!error!!! )

In [4]:
import gc
del df_user
del df_month3
gc.collect()

0

Use pd.Merge to connect data based on user_id and determine if the data is decreasing:

In [5]:
def user_action_id_check():
    df_user = pd.read_csv('data/JData_User.csv')
    df_user = df_user.loc[:,'user_id'].to_frame()
    df_month2 = pd.read_csv('data/JData_Action_201602.csv')
    print ('Is action of Feb. from User file? ', len(df_month2) == len(pd.merge(df_user,df_month2)))
    df_month3 = pd.read_csv('data/JData_Action_201603.csv')
    print ('Is action of Mar. from User file? ', len(df_month3) == len(pd.merge(df_user,df_month3)))
    df_month4 = pd.read_csv('data/JData_Action_201604.csv')
    print ('Is action of Apr. from User file? ', len(df_month4) == len(pd.merge(df_user,df_month4)))
    del df_user,df_month2,df_month3,df_month4
    gc.collect()
user_action_id_check()

Is action of Feb. from User file?  True
Is action of Mar. from User file?  True
Is action of Apr. from User file?  True


In [6]:
#Conclusion: The users in the User dataset and the users in the Interaction dataset are identical

#Based on the comparison of the data volumes before and after merge, we can tell that 
#it is guaranteed that the user ID in Action is a subset of the ID in User

# # Check for duplication

 The possible explanation for this is that duplicate data is meaningful, e.g. users buying multiple items at the same time, adding multiple quantities of items to the cart at the same time, etc...

In [16]:
def deduplicate(filepath, filename, newpath):
    df_file = pd.read_csv(filepath)       
    before = df_file.shape[0]
    df_file.drop_duplicates(inplace=True)
    after = df_file.shape[0]
    n_dup = before-after
    if n_dup != 0:
        print ('No. of duplicate records for ' + filename + ' is: ' + str(n_dup))
        df_file.to_csv(newpath, index=None)
    else:
        print ('no duplicate records in ' + filename)
    del df_file
    gc.collect()

Check each data file for duplicate data

In [17]:
deduplicate('data/JData_Action_201602.csv', 'Feb. action', 'data/JData_Action_201602_dedup.csv')
deduplicate('data/JData_Action_201603.csv', 'Mar. action', 'data/JData_Action_201603_dedup.csv')
deduplicate('data/JData_Action_201604.csv', 'Apr. action', 'data/JData_Action_201604_dedup.csv')
deduplicate('data/JData_Comment.csv', 'Comment', 'data/JData_Comment_dedup.csv')
deduplicate('data/JData_Product.csv', 'Product', 'data/JData_Product_dedup.csv')
deduplicate('data/JData_User.csv', 'User', 'data/JData_User_dedup.csv')

No. of duplicate records for Feb. action is: 2756093
No. of duplicate records for Mar. action is: 7085037
No. of duplicate records for Apr. action is: 3672710
no duplicate records in Comment
no duplicate records in Product
no duplicate records in User


In [ ]:
Repeat the steps of data analysis

In [20]:
df_month3 = pd.read_csv('data/JData_Action_201603.csv')
IsDuplicated = df_month3.duplicated() 
df_d=df_month3[IsDuplicated]
# Most of the duplicates were found to be generated by browsing (1), or clicking (6)
display(df_d.groupby('type').count())
del df_month3,df_d
gc.collect()

,user_id,sku_id,time,model_id,cate,brand
type,,,,,,
1,4659616,4659616,4659616,0,4659616,4659616
2,1406,1406,1406,0,1406,1406
3,2360,2360,2360,0,2360,2360
4,39,39,39,0,39,39
5,4475,4475,4475,0,4475,4475
6,2417141,2417141,2417141,2355765,2417141,2417141


0

# # Build User Table

In order to be able to perform the above cleaning, a simple user behavioural profile is first constructed here, corresponding to the table user_table

user_table features include:
user_id(user id), age(age), sex(gender),
user_lv_cd(user_level), browse_num(number of views),
addcart_num(addcart_num), delcart_num(deletecart_num),
buy_num(number of purchases), favor_num(number of favorites),
click_num(number of clicks),buy_addcart_ratio(buy addcart conversion rate),
buy_browse_ratio(buy_browse_ratio),
buy_click_ratio (buy_click_ratio),
buy_favor_ratio (buy_favor_ratio)

(Conditions Known)

In [7]:
# Define the file name
ACTION_201602_FILE = "data/JData_Action_201602.csv"
ACTION_201603_FILE = "data/JData_Action_201603.csv"
ACTION_201604_FILE = "data/JData_Action_201604.csv"
COMMENT_FILE = "data/JData_Comment.csv"
PRODUCT_FILE = "data/JData_Product.csv"
USER_FILE = "data/JData_User.csv"
USER_TABLE_FILE = "data/User_table.csv"

# # Define functions to count the frequency of user actions

In [8]:
# Importing relevant information packages
import pandas as pd
import numpy as np
from collections import Counter
# Using functions: Statistics for each user group
def add_type_count(group):
    behavior_type = group.type.astype(int)
    # Dividing customers into different types
    type_cnt = Counter(behavior_type)
    
    # 1: Browse 2: Add to Order 3: Delete
    # 4: Buy 5: Bookmark 6: Click
    group['browse_num'] = type_cnt[1]
    group['addcart_num'] = type_cnt[2]
    group['delcart_num'] = type_cnt[3]
    group['buy_num'] = type_cnt[4]
    group['favor_num'] = type_cnt[5]
    group['click_num'] = type_cnt[6]

    return group[['user_id', 'browse_num', 'addcart_num',
                  'delcart_num', 'buy_num', 'favor_num',
                  'click_num']]

# # User behaviour data read in chunks

Due to the large volume of user behaviour data, a single read may cause a Memory Error, so pandas is used to read in chunks.

In [9]:
#Statistics on action data
#Adjust chunk_size 
def get_from_action_data(fname, chunk_size=50000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            # Read only the user_id and type fields
            chunk = reader.get_chunk(chunk_size)[["user_id", "type"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    # Splice blocks into pandas dataframe format
    df_ac = pd.concat(chunks, ignore_index=True)
    # grouped by user_id, with statistics for each group, as_index means return data without index
    df_ac = df_ac.groupby(['user_id'], as_index=False).apply(add_type_count)
    # Discard duplicate rows
    df_ac = df_ac.drop_duplicates('user_id')
    return df_ac

## February user data processing view

In [10]:
df_ac = get_from_action_data(fname = ACTION_201602_FILE, chunk_size=50000)
display(df_ac.head(10))
del df_ac
gc.collect()

Iteration is stopped


,user_id,browse_num,addcart_num,delcart_num,buy_num,favor_num,click_num
0,266079,108,10,6,1,0,270
2,200719,151,12,6,5,7,233
4,263587,49,0,0,0,0,0
5,296130,71,0,0,0,0,181
8,217892,372,0,0,0,1,548
9,283139,29,9,5,3,0,4
11,243907,54,0,1,0,0,100
13,209278,47,2,0,0,0,89
20,217567,5,0,0,0,2,10
22,207684,142,10,10,1,0,217


0

# #Define function to aggregate all data

In [12]:
# Aggregate the statistics of individual action data
def merge_action_data():
    df_ac = []
    df_ac.append(get_from_action_data(fname=ACTION_201602_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201603_FILE))
    df_ac.append(get_from_action_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # User summation of statistics in different action tables
    df_ac = df_ac.groupby(['user_id'], as_index=False).sum()
    
    #　Constructing the conversion rate field
    df_ac['buy_addcart_ratio'] = df_ac['buy_num'] / df_ac['addcart_num']
    df_ac['buy_browse_ratio'] = df_ac['buy_num'] / df_ac['browse_num']
    df_ac['buy_click_ratio'] = df_ac['buy_num'] / df_ac['click_num']
    df_ac['buy_favor_ratio'] = df_ac['buy_num'] / df_ac['favor_num']
    
    # Set the conversion rate field greater than 1 to 1 (100%)
    print((df_ac['buy_addcart_ratio'] > 1.).sum())
    df_ac.loc[df_ac['buy_addcart_ratio'] > 1., 'buy_addcart_ratio'] = 1.
    df_ac.loc[df_ac['buy_browse_ratio'] > 1., 'buy_browse_ratio'] = 1.
    df_ac.loc[df_ac['buy_click_ratio'] > 1., 'buy_click_ratio'] = 1.
    df_ac.loc[df_ac['buy_favor_ratio'] > 1., 'buy_favor_ratio'] = 1.

    return df_ac